# Titanic Disaster
<p> In this notebook I participate in <a html='https://www.kaggle.com/competitions/titanic/'>Kaggle's Titanic Competition</a> in order to predict which passengers survived the shipwreck</p> 

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
import tensorflow as tf

## Import dataset

In [2]:
train = pd.read_csv('train.csv')
X_train = train.drop(columns=['Survived'], axis=1)
y_train = train.iloc[:,1].values

test_set = pd.read_csv('test.csv')
y_test = pd.read_csv('gender_submission.csv')
y_test = y_test.iloc[:,-1].values

## Preprocoessing the data

In [3]:
# Dropping unnecessary columns
X_train = X_train.drop(columns=['PassengerId','Name', 'Ticket'])
X_test = test_set.drop(columns=['PassengerId','Name', 'Ticket'])

In [4]:
# Dealing with nulls
print('Training data:', X_train.isnull().sum(),'', sep='\n')
print('Testing data:',X_test.isnull().sum(),sep='\n')

Training data:
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

Testing data:
Pclass        0
Sex           0
Age          86
SibSp         0
Parch         0
Fare          1
Cabin       327
Embarked      0
dtype: int64


In [5]:
# Dropping the 'Cabin' model since it has more null values than non-null
X_train = X_train.drop(columns=['Cabin'])
# Filling missing values in the 'Age' column with the mean age
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
# Filling missing values in the 'Embarked' column with the mode age
X_train['Embarked'].fillna(X_train['Embarked'].mode()[0], inplace=True)
X_train.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [6]:
# Now do the same for the test set
X_test = X_test.drop(columns=['Cabin'])
X_test['Age'].fillna(X_test['Age'].mean(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].mean(), inplace=True)
X_test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### Encoding Data

In [7]:
columns_to_encode = ['Sex', 'Pclass', 'Embarked']
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),columns_to_encode)], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

### Feature Scaling

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Training different models
<p> Trainig various classification models in search of the one that gives the best results</p>

### Logistic regression

In [9]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
# Perform 5-fold cross-validation
reg_cv_scores = cross_val_score(log_reg, X_train, y_train, cv=10, scoring='accuracy')
# Print the mean and standard deviation of the cross-validation scores
print(f"Mean Accuracy: {reg_cv_scores.mean():.4f}")
print(f"Standard Deviation: {reg_cv_scores.std():.4f}")

Mean Accuracy: 0.7980
Standard Deviation: 0.0259


### K-Nearest Neighbors

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
# Perform 5-fold cross-validation
knn_cv_scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
# Print the mean and standard deviation of the cross-validation scores
print(f"Mean Accuracy: {knn_cv_scores.mean():.4f}")
print(f"Standard Deviation: {knn_cv_scores.std():.4f}")

Mean Accuracy: 0.7936
Standard Deviation: 0.0336


### SVM

In [11]:
from sklearn.svm import SVC
svc = SVC(kernel='linear', random_state=0)
# Perform 5-fold cross-validation
svc_cv_scores = cross_val_score(svc, X_train, y_train, cv=10, scoring='accuracy')
# Print the mean and standard deviation of the cross-validation scores
print(f"Mean Accuracy: {svc_cv_scores.mean():.4f}")
print(f"Standard Deviation: {svc_cv_scores.std():.4f}")

Mean Accuracy: 0.7867
Standard Deviation: 0.0286


### Kernel SVM 

In [12]:
from sklearn.svm import SVC
kernel = SVC(kernel='rbf', random_state=0)
# Perform 10-fold cross-validation
kernel_cv_scores = cross_val_score(kernel, X_train, y_train, cv=10, scoring='accuracy')
# Print the mean and standard deviation of the cross-validation scores
print(f"Mean Accuracy: {kernel_cv_scores.mean():.4f}")
print(f"Standard Deviation: {kernel_cv_scores.std():.4f}")

Mean Accuracy: 0.8260
Standard Deviation: 0.0319


### Naive-Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb_cv_scores = cross_val_score(gnb, X_train, y_train, cv=10, scoring='accuracy')
print(f"Mean Accuracy: {gnb_cv_scores.mean():.4f}")
print(f"Standard Deviation: {gnb_cv_scores.std():.4f}")

Mean Accuracy: 0.7857
Standard Deviation: 0.0282


### Decision Tree Classification

In [14]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy', random_state=0)

dtc_cv_scores = cross_val_score(dtc, X_train, y_train, cv=10, scoring='accuracy')
print(f"Mean Accuracy: {dtc_cv_scores.mean():.4f}")
print(f"Standard Deviation: {dtc_cv_scores.std():.4f}")

Mean Accuracy: 0.7801
Standard Deviation: 0.0467


### Random Forest Classification

In [15]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=15, criterion='entropy')

rfc_cv_scores = cross_val_score(rfc, X_train, y_train, cv=10, scoring='accuracy')
print(f"Mean Accuracy: {rfc_cv_scores.mean():.4f}")
print(f"Standard Deviation: {rfc_cv_scores.std():.4f}")

Mean Accuracy: 0.8115
Standard Deviation: 0.0506


### XGBoosting

In [16]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb_cv_scores = cross_val_score(xgb, X_train, y_train, cv=10, scoring='accuracy')
print(f"Mean Accuracy: {xgb_cv_scores.mean():.4f}")
print(f"Standard Deviation: {xgb_cv_scores.std():.4f}")

Mean Accuracy: 0.8171
Standard Deviation: 0.0395


### CatBoost

In [17]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier()
cat.fit(X_train, y_train)
cat_cv_scores = cross_val_score(cat, X_train, y_train, cv=10, scoring='accuracy')
print(f"Mean Accuracy: {cat_cv_scores.mean():.4f}")
print(f"Standard Deviation: {cat_cv_scores.std():.4f}")

Learning rate set to 0.009807
0:	learn: 0.6875901	total: 76.3ms	remaining: 1m 16s
1:	learn: 0.6812923	total: 77.5ms	remaining: 38.6s
2:	learn: 0.6743466	total: 78.7ms	remaining: 26.1s
3:	learn: 0.6681772	total: 79.9ms	remaining: 19.9s
4:	learn: 0.6624666	total: 81.1ms	remaining: 16.1s
5:	learn: 0.6563153	total: 82.3ms	remaining: 13.6s
6:	learn: 0.6506126	total: 83.4ms	remaining: 11.8s
7:	learn: 0.6451794	total: 84.7ms	remaining: 10.5s
8:	learn: 0.6396678	total: 85.9ms	remaining: 9.46s
9:	learn: 0.6349542	total: 87.1ms	remaining: 8.62s
10:	learn: 0.6296884	total: 88.3ms	remaining: 7.94s
11:	learn: 0.6244800	total: 89.5ms	remaining: 7.37s
12:	learn: 0.6196676	total: 90.9ms	remaining: 6.9s
13:	learn: 0.6144110	total: 92.1ms	remaining: 6.49s
14:	learn: 0.6117873	total: 92.7ms	remaining: 6.09s
15:	learn: 0.6068837	total: 93.9ms	remaining: 5.77s
16:	learn: 0.6020477	total: 95.1ms	remaining: 5.5s
17:	learn: 0.5980093	total: 95.9ms	remaining: 5.23s
18:	learn: 0.5931440	total: 97.1ms	remaining:

164:	learn: 0.3909699	total: 273ms	remaining: 1.38s
165:	learn: 0.3907401	total: 275ms	remaining: 1.38s
166:	learn: 0.3906845	total: 275ms	remaining: 1.37s
167:	learn: 0.3900796	total: 277ms	remaining: 1.37s
168:	learn: 0.3898337	total: 278ms	remaining: 1.36s
169:	learn: 0.3893810	total: 279ms	remaining: 1.36s
170:	learn: 0.3891993	total: 280ms	remaining: 1.36s
171:	learn: 0.3888954	total: 281ms	remaining: 1.35s
172:	learn: 0.3885698	total: 282ms	remaining: 1.35s
173:	learn: 0.3881529	total: 284ms	remaining: 1.34s
174:	learn: 0.3878695	total: 285ms	remaining: 1.34s
175:	learn: 0.3876926	total: 286ms	remaining: 1.34s
176:	learn: 0.3872882	total: 287ms	remaining: 1.33s
177:	learn: 0.3869319	total: 288ms	remaining: 1.33s
178:	learn: 0.3865950	total: 290ms	remaining: 1.33s
179:	learn: 0.3862440	total: 291ms	remaining: 1.32s
180:	learn: 0.3860008	total: 292ms	remaining: 1.32s
181:	learn: 0.3858019	total: 293ms	remaining: 1.32s
182:	learn: 0.3855829	total: 295ms	remaining: 1.31s
183:	learn: 

371:	learn: 0.3486152	total: 521ms	remaining: 880ms
372:	learn: 0.3485041	total: 522ms	remaining: 878ms
373:	learn: 0.3483723	total: 524ms	remaining: 876ms
374:	learn: 0.3482759	total: 525ms	remaining: 874ms
375:	learn: 0.3480990	total: 526ms	remaining: 872ms
376:	learn: 0.3479980	total: 527ms	remaining: 871ms
377:	learn: 0.3479365	total: 528ms	remaining: 869ms
378:	learn: 0.3478377	total: 530ms	remaining: 868ms
379:	learn: 0.3476417	total: 531ms	remaining: 866ms
380:	learn: 0.3475745	total: 532ms	remaining: 864ms
381:	learn: 0.3474881	total: 533ms	remaining: 862ms
382:	learn: 0.3473471	total: 534ms	remaining: 860ms
383:	learn: 0.3472968	total: 535ms	remaining: 859ms
384:	learn: 0.3471500	total: 537ms	remaining: 857ms
385:	learn: 0.3471161	total: 538ms	remaining: 856ms
386:	learn: 0.3470237	total: 539ms	remaining: 855ms
387:	learn: 0.3468650	total: 541ms	remaining: 853ms
388:	learn: 0.3466443	total: 542ms	remaining: 852ms
389:	learn: 0.3465154	total: 544ms	remaining: 850ms
390:	learn: 

602:	learn: 0.3229508	total: 799ms	remaining: 526ms
603:	learn: 0.3228780	total: 801ms	remaining: 525ms
604:	learn: 0.3227785	total: 802ms	remaining: 524ms
605:	learn: 0.3226567	total: 803ms	remaining: 522ms
606:	learn: 0.3226151	total: 805ms	remaining: 521ms
607:	learn: 0.3225033	total: 806ms	remaining: 520ms
608:	learn: 0.3223496	total: 807ms	remaining: 518ms
609:	learn: 0.3222088	total: 809ms	remaining: 517ms
610:	learn: 0.3221065	total: 810ms	remaining: 516ms
611:	learn: 0.3220136	total: 811ms	remaining: 514ms
612:	learn: 0.3219625	total: 812ms	remaining: 513ms
613:	learn: 0.3218256	total: 814ms	remaining: 512ms
614:	learn: 0.3217602	total: 815ms	remaining: 510ms
615:	learn: 0.3216917	total: 816ms	remaining: 509ms
616:	learn: 0.3216514	total: 817ms	remaining: 507ms
617:	learn: 0.3216168	total: 818ms	remaining: 506ms
618:	learn: 0.3215696	total: 820ms	remaining: 504ms
619:	learn: 0.3215646	total: 820ms	remaining: 503ms
620:	learn: 0.3213641	total: 821ms	remaining: 501ms
621:	learn: 

805:	learn: 0.3019517	total: 1.05s	remaining: 252ms
806:	learn: 0.3018654	total: 1.05s	remaining: 251ms
807:	learn: 0.3017504	total: 1.05s	remaining: 250ms
808:	learn: 0.3016862	total: 1.05s	remaining: 248ms
809:	learn: 0.3015497	total: 1.05s	remaining: 247ms
810:	learn: 0.3014551	total: 1.05s	remaining: 246ms
811:	learn: 0.3014337	total: 1.05s	remaining: 244ms
812:	learn: 0.3012999	total: 1.06s	remaining: 243ms
813:	learn: 0.3010264	total: 1.06s	remaining: 242ms
814:	learn: 0.3009958	total: 1.06s	remaining: 240ms
815:	learn: 0.3008588	total: 1.06s	remaining: 239ms
816:	learn: 0.3006892	total: 1.06s	remaining: 238ms
817:	learn: 0.3005905	total: 1.06s	remaining: 236ms
818:	learn: 0.3003920	total: 1.06s	remaining: 235ms
819:	learn: 0.3002918	total: 1.06s	remaining: 234ms
820:	learn: 0.3001771	total: 1.07s	remaining: 232ms
821:	learn: 0.3000952	total: 1.07s	remaining: 231ms
822:	learn: 0.2999958	total: 1.07s	remaining: 230ms
823:	learn: 0.2998671	total: 1.07s	remaining: 228ms
824:	learn: 

...

123:	learn: 0.4205062	total: 153ms	remaining: 1.08s
124:	learn: 0.4196749	total: 155ms	remaining: 1.08s
125:	learn: 0.4188159	total: 156ms	remaining: 1.08s
126:	learn: 0.4185998	total: 157ms	remaining: 1.08s
127:	learn: 0.4181857	total: 158ms	remaining: 1.08s
128:	learn: 0.4175145	total: 159ms	remaining: 1.07s
129:	learn: 0.4171995	total: 160ms	remaining: 1.07s
130:	learn: 0.4164999	total: 161ms	remaining: 1.07s
131:	learn: 0.4159561	total: 162ms	remaining: 1.07s
132:	learn: 0.4153563	total: 164ms	remaining: 1.07s
133:	learn: 0.4148268	total: 165ms	remaining: 1.06s
134:	learn: 0.4141722	total: 166ms	remaining: 1.06s
135:	learn: 0.4137074	total: 167ms	remaining: 1.06s
136:	learn: 0.4130272	total: 168ms	remaining: 1.06s
137:	learn: 0.4128033	total: 170ms	remaining: 1.06s
138:	learn: 0.4124712	total: 171ms	remaining: 1.06s
139:	learn: 0.4119330	total: 172ms	remaining: 1.06s
140:	learn: 0.4111771	total: 173ms	remaining: 1.05s
141:	learn: 0.4106527	total: 175ms	remaining: 1.05s
142:	learn: 

281:	learn: 0.3688872	total: 341ms	remaining: 869ms
282:	learn: 0.3686732	total: 343ms	remaining: 868ms
283:	learn: 0.3684990	total: 344ms	remaining: 867ms
284:	learn: 0.3683695	total: 345ms	remaining: 865ms
285:	learn: 0.3681494	total: 346ms	remaining: 864ms
286:	learn: 0.3678442	total: 347ms	remaining: 863ms
287:	learn: 0.3677063	total: 349ms	remaining: 862ms
288:	learn: 0.3675916	total: 350ms	remaining: 861ms
289:	learn: 0.3674215	total: 351ms	remaining: 860ms
290:	learn: 0.3671413	total: 352ms	remaining: 858ms
291:	learn: 0.3669543	total: 353ms	remaining: 857ms
292:	learn: 0.3667669	total: 355ms	remaining: 856ms
293:	learn: 0.3664842	total: 356ms	remaining: 855ms
294:	learn: 0.3664075	total: 357ms	remaining: 854ms
295:	learn: 0.3663301	total: 358ms	remaining: 852ms
296:	learn: 0.3662283	total: 360ms	remaining: 851ms
297:	learn: 0.3661750	total: 361ms	remaining: 851ms
298:	learn: 0.3660690	total: 362ms	remaining: 850ms
299:	learn: 0.3659730	total: 364ms	remaining: 849ms
300:	learn: 

463:	learn: 0.3416462	total: 562ms	remaining: 649ms
464:	learn: 0.3415404	total: 563ms	remaining: 648ms
465:	learn: 0.3415106	total: 564ms	remaining: 647ms
466:	learn: 0.3413376	total: 565ms	remaining: 645ms
467:	learn: 0.3411624	total: 567ms	remaining: 644ms
468:	learn: 0.3410470	total: 568ms	remaining: 643ms
469:	learn: 0.3409872	total: 569ms	remaining: 641ms
470:	learn: 0.3408955	total: 570ms	remaining: 640ms
471:	learn: 0.3408768	total: 571ms	remaining: 639ms
472:	learn: 0.3407710	total: 572ms	remaining: 637ms
473:	learn: 0.3405362	total: 573ms	remaining: 636ms
474:	learn: 0.3403995	total: 574ms	remaining: 635ms
475:	learn: 0.3402668	total: 576ms	remaining: 634ms
476:	learn: 0.3401863	total: 577ms	remaining: 633ms
477:	learn: 0.3400680	total: 578ms	remaining: 632ms
478:	learn: 0.3398464	total: 580ms	remaining: 630ms
479:	learn: 0.3396871	total: 581ms	remaining: 629ms
480:	learn: 0.3393503	total: 582ms	remaining: 628ms
481:	learn: 0.3392655	total: 584ms	remaining: 627ms
482:	learn: 

723:	learn: 0.3136266	total: 872ms	remaining: 332ms
724:	learn: 0.3136049	total: 873ms	remaining: 331ms
725:	learn: 0.3135039	total: 874ms	remaining: 330ms
726:	learn: 0.3134867	total: 876ms	remaining: 329ms
727:	learn: 0.3134157	total: 877ms	remaining: 328ms
728:	learn: 0.3132072	total: 878ms	remaining: 326ms
729:	learn: 0.3131570	total: 879ms	remaining: 325ms
730:	learn: 0.3129379	total: 880ms	remaining: 324ms
731:	learn: 0.3129071	total: 881ms	remaining: 323ms
732:	learn: 0.3127954	total: 882ms	remaining: 321ms
733:	learn: 0.3127107	total: 884ms	remaining: 320ms
734:	learn: 0.3126122	total: 885ms	remaining: 319ms
735:	learn: 0.3125078	total: 886ms	remaining: 318ms
736:	learn: 0.3123250	total: 887ms	remaining: 317ms
737:	learn: 0.3120515	total: 889ms	remaining: 315ms
738:	learn: 0.3119618	total: 890ms	remaining: 314ms
739:	learn: 0.3117132	total: 891ms	remaining: 313ms
740:	learn: 0.3115081	total: 892ms	remaining: 312ms
741:	learn: 0.3114332	total: 894ms	remaining: 311ms
742:	learn: 

903:	learn: 0.2937970	total: 1.09s	remaining: 115ms
904:	learn: 0.2936935	total: 1.09s	remaining: 114ms
905:	learn: 0.2936008	total: 1.09s	remaining: 113ms
906:	learn: 0.2935529	total: 1.09s	remaining: 112ms
907:	learn: 0.2933393	total: 1.09s	remaining: 111ms
908:	learn: 0.2932372	total: 1.09s	remaining: 109ms
909:	learn: 0.2931429	total: 1.09s	remaining: 108ms
910:	learn: 0.2930968	total: 1.09s	remaining: 107ms
911:	learn: 0.2929789	total: 1.1s	remaining: 106ms
912:	learn: 0.2928789	total: 1.1s	remaining: 105ms
913:	learn: 0.2927664	total: 1.1s	remaining: 103ms
914:	learn: 0.2927163	total: 1.1s	remaining: 102ms
915:	learn: 0.2924977	total: 1.1s	remaining: 101ms
916:	learn: 0.2923357	total: 1.1s	remaining: 99.8ms
917:	learn: 0.2922411	total: 1.1s	remaining: 98.6ms
918:	learn: 0.2921215	total: 1.1s	remaining: 97.4ms
919:	learn: 0.2920550	total: 1.1s	remaining: 96.1ms
920:	learn: 0.2919230	total: 1.11s	remaining: 94.9ms
921:	learn: 0.2917392	total: 1.11s	remaining: 93.7ms
922:	learn: 0.2

### Artificial Neural Network

In [18]:
# Initializing ANN
ann = tf.keras.models.Sequential()

# Adding hidden layers
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compiling ANN
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit ANN
ann.fit(X_train, y_train, batch_size=32, epochs=50)



Epoch 1/50


28/28 [==============================] - 1s 677us/step - loss: 0.7612 - accuracy: 0.3895
Epoch 2/50
28/28 [==============================] - 0s 590us/step - loss: 0.7055 - accuracy: 0.4590
Epoch 3/50
28/28 [==============================] - 0s 1ms/step - loss: 0.6708 - accuracy: 0.6498
Epoch 4/50
28/28 [==============================] - 0s 641us/step - loss: 0.6475 - accuracy: 0.7407
Epoch 5/50
28/28 [==============================] - 0s 930us/step - loss: 0.6286 - accuracy: 0.7576
Epoch 6/50
28/28 [==============================] - 0s 730us/step - loss: 0.6099 - accuracy: 0.7755
Epoch 7/50
28/28 [==============================] - 0s 690us/step - loss: 0.5895 - accuracy: 0.7733
Epoch 8/50
28/28 [==============================] - 0s 613us/step - loss: 0.5664 - accuracy: 0.7800
Epoch 9/50
28/28 [==============================] - 0s 614us/step - loss: 0.5407 - accuracy: 0.7845
Epoch 10/50
28/28 [==============================] - 0s 714us/step - loss: 0.5151 - accuracy: 0.79

## Training Results

<p> After comparing the accuracies of each model, I conclude that the artificial neural network is the best model. Now I will use it to predict the test set and create a pandas dataframe. After creating the dataframe I can export it into a csv file in order to submit it to Kaggle.</p>

### Predicting with an Artificial Neural Network

In [19]:
# Predict testing data set
y_pred = ann.predict(X_test)

# Convert probability to binary 
y_pred[y_pred<.5] = 0
y_pred[y_pred>=.5] = 1

# Reshape
y_pred = y_pred.reshape(418)

14/14 [==============================] - 0s 1ms/step


In [20]:
titanic_submission = pd.DataFrame({'PassengerId':test_set['PassengerId'], 'Survived':y_pred}).astype('int64')
titanic_submission.to_csv('titanic_submission.csv', index=False)

In [21]:
titanic_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
